In [49]:
# !pip install catboost

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data.csv')
df['date'] = pd.to_datetime(df['date'])

print(df.shape)
print('Number of trading days:', (df['date'].max() - df['date'].min()).days)
print('Number of tickers:', df['ticker'].nunique())

(451113, 4)
Number of trading days: 2996
Number of tickers: 248


## Data Processing: Generate trading indicators

In [8]:
def compute_vwap(df, look_back=5):
    '''
    Volume-weighted Average Price
    '''
    adj_close = df['last']
    vol = df['volume']
    vol_weighted_close = adj_close * vol
    vwap = vol_weighted_close.rolling(window=look_back).sum() / vol.rolling(window=look_back).sum()
#     vwap =  .adj_close.rolling(window=look_back).sum()
#     vwap = (adj_close * vol).cumsum() / vol.cumsum()
    signal = adj_close - vwap
    return df.assign(vwap=vwap, vwap_signal=signal)
     
    
    
def compute_bb(df, look_back=5):
    '''
    Bollinger Band
    '''
    def compute_sma(adj_close, look_back):
        sma = adj_close.rolling(window=look_back, center=False).mean()
        ratio = adj_close/sma
        return sma, ratio
    
    adj_close = df['last']
    sma, _ = compute_sma(adj_close, look_back)
    rolling_std = adj_close.rolling(window=look_back, center=False).std()
    
    upper = sma + 2*rolling_std
    lower = sma - 2*rolling_std
    signal = (adj_close - sma) /(2*rolling_std)
    return df.assign(upper_bb=upper, lower_bb=lower, bb_signal=signal)



def compute_so(df, look_back=5, shift=5):
    '''
    Stochastic Oscillator
    '''
    adj_close = df['last']
    recent_highest = adj_close.rolling(window=look_back, center=False).max().shift(shift)
    recent_lowest = adj_close.rolling(window=look_back, center=False).min().shift(shift)
    signal = (adj_close - recent_lowest) / (recent_highest - recent_lowest)
    return df.assign(so_signal=signal)



def compute_macd(df, 
                 fast_period=12, 
                 slow_period=26, 
                 norm_period=20, 
                 signal_period=9, 
                 smooth_period=5, 
                 smooth=False, 
                 shift=0):
    '''
    Moving Average Convergence Divergence
    '''
    adj_close = df['last']
    ema_fast = adj_close.ewm(span=fast_period).mean().shift(shift)
    ema_slow = adj_close.ewm(span=slow_period).mean().shift(shift)
    
    macd = ema_fast - ema_slow
    recent_max_macd = macd.rolling(window=norm_period, center=False).max()
    recent_min_macd = macd.rolling(window=norm_period, center=False).min()
    
    macd_normalized = (macd - recent_min_macd) / (recent_max_macd - recent_min_macd)
    macd_final = macd_normalized.ewm(span=smooth_period).mean() if smooth else macd_normalized
    signal = macd_final.ewm(span=signal_period).mean()
    
    return df.assign(macd_last=macd_final, macd_signal=signal)



def compute_momentum(df, look_back=15):
    '''
    Momentum
    '''
    adj_close = df['last']
    momentum = adj_close / adj_close.shift(look_back) - 1
    return df.assign(momentum=momentum)


In [9]:
df = df.sort_values(by='date')
df = df.groupby('ticker', group_keys=False).apply(compute_vwap)
df = df.groupby('ticker', group_keys=False).apply(compute_macd)
df = df.groupby('ticker', group_keys=False).apply(compute_bb)
df = df.groupby('ticker', group_keys=False).apply(compute_momentum)
df = df.groupby('ticker', group_keys=False).apply(compute_so)

In [16]:
df_merged = pd.pivot_table(df, index=['date'], 
                           values=[c for c in df.columns if c not in {'ticker', 'date'}], 
                           columns=['ticker'])

# add datetime features
df_merged['DT_quarter'] = df_merged.index.quarter
df_merged['DT_month'] = df_merged.index.month
df_merged['DT_weekofyear'] = df_merged.index.weekofyear
df_merged['DT_is_month_start'] = df_merged.index.is_month_start.astype(int)
df_merged['DT_is_month_end'] = df_merged.index.is_month_end.astype(int)
df_merged['DT_day'] = df_merged.index.day
df_merged['DT_dayofyear'] = df_merged.index.dayofyear
df_merged['DT_weekday'] = df_merged.index.weekday

all_tickers = list(set([j for (i, j) in df_merged.columns if (not i.startswith('DT')) and (not i in {'last', 'volume'})]))
dt_features = [i for (i, j) in df_merged.columns if i.startswith('DT')]
indicator_features = list(set([i for (i, j) in df_merged.columns if (not i.startswith('DT')) and (not i in {'last', 'volume'})]))
raw_features = ['last', 'volume']

(489, 20)
(121, 22)


## Model Training: Feed technical indicators to classification model 

In [51]:
dates = df_merged.index
start_date, end_date = dates.min(), dates.max()
print(start_date, end_date)

fr_threshold = 0.02


pools = {
    s: Pool(
        data=splits[s][feature_names],
        label=splits[s]['label'],
#         cat_features=dt_features,
    ) for s in ['train', 'val']
}

params = {
    "task_type": "CPU",
    "devices": '0',
    "max_ctr_complexity": 1,
    "boosting_type": "Plain",
    "use_best_model": True,
    "iterations": 1000,
    "eval_metric": "BalancedErrorRate",
    "early_stopping_rounds": 200,
    "objective": "Logloss",
    "depth": 6,
}

model = CatBoostClassifier(**params)

2013-01-04 00:00:00 2021-03-19 00:00:00


In [ ]:
for ticker in df['ticker'].unique():
    df_dataset = df_merged.loc[start_date:end_date][[(f, ticker) for f in indicator_features+raw_features]]
    for c in dt_features:
        df_dataset[c] = df_merged.loc[start_date:end_date][c]

    df_dataset.columns = df_dataset.columns.get_level_values(0)
    print(df_dataset.shape)

    df_dataset['next_day_return'] = df_dataset['last'].shift(-1) / df_dataset['last'] - 1
    df_dataset['label'] = df_dataset.apply(lambda r: 1 if r['next_day_return'] > fr_threshold else (-1 if r['next_day_return'] < -fr_threshold else np.nan), axis=1)
    df_dataset = df_dataset.dropna()
    print(df_dataset.shape)
    
    train_size = 0.7
    trading_days = dates.max() - dates.min()
    last_train_date = dates.min() + trading_days * train_size
    train_df, val_df = df_dataset.loc[:last_train_date], df_dataset.loc[last_train_date:]
    splits = {'train': train_df, 'val': val_df}
    feature_names = [c for c in df_dataset.columns if c not in {'label', 'next_day_return'}]

    model.fit(pools['train'], eval_set=pools['val'], use_best_model=True, verbose=1)

(2005, 20)
(512, 22)
Learning rate set to 0.023785
0:	learn: 0.3739265	test: 0.4952975	best: 0.4952975 (0)	total: 3.08ms	remaining: 3.08s
1:	learn: 0.3365400	test: 0.4528726	best: 0.4528726 (1)	total: 6.49ms	remaining: 3.24s
2:	learn: 0.3328035	test: 0.5026579	best: 0.4528726 (1)	total: 9.55ms	remaining: 3.17s
3:	learn: 0.2860446	test: 0.4603353	best: 0.4528726 (1)	total: 12.7ms	remaining: 3.16s
4:	learn: 0.2739678	test: 0.4534860	best: 0.4528726 (1)	total: 15.9ms	remaining: 3.17s
5:	learn: 0.2777044	test: 0.5020446	best: 0.4528726 (1)	total: 19.1ms	remaining: 3.16s
6:	learn: 0.2748142	test: 0.5318953	best: 0.4528726 (1)	total: 22.1ms	remaining: 3.14s
7:	learn: 0.2834847	test: 0.5194234	best: 0.4528726 (1)	total: 25.4ms	remaining: 3.15s
8:	learn: 0.2705615	test: 0.5057248	best: 0.4528726 (1)	total: 28.7ms	remaining: 3.16s
9:	learn: 0.2741329	test: 0.5337354	best: 0.4528726 (1)	total: 32.1ms	remaining: 3.18s
10:	learn: 0.2634187	test: 0.5113474	best: 0.4528726 (1)	total: 35.8ms	remainin

121:	learn: 0.0321429	test: 0.5232059	best: 0.4528726 (1)	total: 500ms	remaining: 3.6s
122:	learn: 0.0357143	test: 0.5232059	best: 0.4528726 (1)	total: 508ms	remaining: 3.62s
123:	learn: 0.0357143	test: 0.5375179	best: 0.4528726 (1)	total: 537ms	remaining: 3.79s
124:	learn: 0.0357143	test: 0.5443672	best: 0.4528726 (1)	total: 545ms	remaining: 3.81s
125:	learn: 0.0285714	test: 0.5306686	best: 0.4528726 (1)	total: 550ms	remaining: 3.81s
126:	learn: 0.0250000	test: 0.5375179	best: 0.4528726 (1)	total: 555ms	remaining: 3.81s
127:	learn: 0.0285714	test: 0.5300552	best: 0.4528726 (1)	total: 561ms	remaining: 3.82s
128:	learn: 0.0285714	test: 0.5369045	best: 0.4528726 (1)	total: 565ms	remaining: 3.82s
129:	learn: 0.0250000	test: 0.5300552	best: 0.4528726 (1)	total: 569ms	remaining: 3.81s
130:	learn: 0.0214286	test: 0.5369045	best: 0.4528726 (1)	total: 573ms	remaining: 3.8s
131:	learn: 0.0178571	test: 0.5232059	best: 0.4528726 (1)	total: 577ms	remaining: 3.8s
132:	learn: 0.0250000	test: 0.53690

27:	learn: 0.1887696	test: 0.4814966	best: 0.4528726 (1)	total: 114ms	remaining: 3.96s
28:	learn: 0.1758464	test: 0.4814966	best: 0.4528726 (1)	total: 121ms	remaining: 4.07s
29:	learn: 0.1829893	test: 0.4951953	best: 0.4528726 (1)	total: 129ms	remaining: 4.16s
30:	learn: 0.1729562	test: 0.5026579	best: 0.4528726 (1)	total: 137ms	remaining: 4.27s
31:	learn: 0.1765277	test: 0.4958086	best: 0.4528726 (1)	total: 141ms	remaining: 4.27s
32:	learn: 0.1729562	test: 0.4964220	best: 0.4528726 (1)	total: 144ms	remaining: 4.23s
33:	learn: 0.1693848	test: 0.5107340	best: 0.4528726 (1)	total: 149ms	remaining: 4.24s
34:	learn: 0.1687036	test: 0.4964220	best: 0.4528726 (1)	total: 152ms	remaining: 4.2s
35:	learn: 0.1579893	test: 0.5107340	best: 0.4528726 (1)	total: 161ms	remaining: 4.32s
36:	learn: 0.1515277	test: 0.5250460	best: 0.4528726 (1)	total: 166ms	remaining: 4.32s
37:	learn: 0.1515277	test: 0.5318953	best: 0.4528726 (1)	total: 171ms	remaining: 4.33s
38:	learn: 0.1479562	test: 0.5318953	best: 0

148:	learn: 0.0142857	test: 0.5151298	best: 0.4528726 (1)	total: 685ms	remaining: 3.91s
149:	learn: 0.0142857	test: 0.5219791	best: 0.4528726 (1)	total: 691ms	remaining: 3.92s
150:	learn: 0.0142857	test: 0.5219791	best: 0.4528726 (1)	total: 700ms	remaining: 3.93s
151:	learn: 0.0107143	test: 0.5225925	best: 0.4528726 (1)	total: 708ms	remaining: 3.95s
152:	learn: 0.0142857	test: 0.5151298	best: 0.4528726 (1)	total: 711ms	remaining: 3.94s
153:	learn: 0.0142857	test: 0.5225925	best: 0.4528726 (1)	total: 716ms	remaining: 3.94s
154:	learn: 0.0107143	test: 0.5225925	best: 0.4528726 (1)	total: 721ms	remaining: 3.93s
155:	learn: 0.0107143	test: 0.5225925	best: 0.4528726 (1)	total: 727ms	remaining: 3.94s
156:	learn: 0.0107143	test: 0.5362911	best: 0.4528726 (1)	total: 732ms	remaining: 3.93s
157:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 735ms	remaining: 3.92s
158:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 740ms	remaining: 3.91s
159:	learn: 0.0107143	test: 0.52

68:	learn: 0.1015277	test: 0.5026579	best: 0.4528726 (1)	total: 375ms	remaining: 5.06s
69:	learn: 0.1050991	test: 0.4951953	best: 0.4528726 (1)	total: 380ms	remaining: 5.04s
70:	learn: 0.1015277	test: 0.5095073	best: 0.4528726 (1)	total: 391ms	remaining: 5.11s
71:	learn: 0.0979562	test: 0.5244326	best: 0.4528726 (1)	total: 399ms	remaining: 5.15s
72:	learn: 0.0943848	test: 0.5175833	best: 0.4528726 (1)	total: 411ms	remaining: 5.22s
73:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 416ms	remaining: 5.2s
74:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 422ms	remaining: 5.2s
75:	learn: 0.0943848	test: 0.5026579	best: 0.4528726 (1)	total: 427ms	remaining: 5.19s
76:	learn: 0.0950661	test: 0.4958086	best: 0.4528726 (1)	total: 430ms	remaining: 5.15s
77:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 436ms	remaining: 5.16s
78:	learn: 0.0843518	test: 0.5169699	best: 0.4528726 (1)	total: 440ms	remaining: 5.13s
79:	learn: 0.0843518	test: 0.5095073	best: 0.

...

152:	learn: 0.0142857	test: 0.5151298	best: 0.4528726 (1)	total: 882ms	remaining: 4.88s
153:	learn: 0.0142857	test: 0.5225925	best: 0.4528726 (1)	total: 895ms	remaining: 4.92s
154:	learn: 0.0107143	test: 0.5225925	best: 0.4528726 (1)	total: 905ms	remaining: 4.93s
155:	learn: 0.0107143	test: 0.5225925	best: 0.4528726 (1)	total: 913ms	remaining: 4.94s
156:	learn: 0.0107143	test: 0.5362911	best: 0.4528726 (1)	total: 918ms	remaining: 4.93s
157:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 923ms	remaining: 4.92s
158:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 927ms	remaining: 4.9s
159:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 931ms	remaining: 4.89s
160:	learn: 0.0107143	test: 0.5219791	best: 0.4528726 (1)	total: 935ms	remaining: 4.87s
161:	learn: 0.0107143	test: 0.5294418	best: 0.4528726 (1)	total: 939ms	remaining: 4.86s
162:	learn: 0.0107143	test: 0.5294418	best: 0.4528726 (1)	total: 944ms	remaining: 4.84s
163:	learn: 0.0107143	test: 0.536

72:	learn: 0.0943848	test: 0.5175833	best: 0.4528726 (1)	total: 379ms	remaining: 4.81s
73:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 385ms	remaining: 4.82s
74:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 394ms	remaining: 4.86s
75:	learn: 0.0943848	test: 0.5026579	best: 0.4528726 (1)	total: 404ms	remaining: 4.91s
76:	learn: 0.0950661	test: 0.4958086	best: 0.4528726 (1)	total: 416ms	remaining: 4.99s
77:	learn: 0.0908134	test: 0.5026579	best: 0.4528726 (1)	total: 420ms	remaining: 4.96s
78:	learn: 0.0843518	test: 0.5169699	best: 0.4528726 (1)	total: 432ms	remaining: 5.04s
79:	learn: 0.0843518	test: 0.5095073	best: 0.4528726 (1)	total: 440ms	remaining: 5.06s
80:	learn: 0.0772089	test: 0.5020446	best: 0.4528726 (1)	total: 445ms	remaining: 5.05s
81:	learn: 0.0736375	test: 0.5020446	best: 0.4528726 (1)	total: 461ms	remaining: 5.16s
82:	learn: 0.0765277	test: 0.5095073	best: 0.4528726 (1)	total: 465ms	remaining: 5.14s
83:	learn: 0.0765277	test: 0.5020446	best: 

169:	learn: 0.0071429	test: 0.5213658	best: 0.4528726 (1)	total: 939ms	remaining: 4.59s
170:	learn: 0.0071429	test: 0.5213658	best: 0.4528726 (1)	total: 945ms	remaining: 4.58s
171:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 955ms	remaining: 4.59s
172:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 964ms	remaining: 4.61s
173:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 971ms	remaining: 4.61s
174:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 978ms	remaining: 4.61s
175:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 982ms	remaining: 4.6s
176:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 987ms	remaining: 4.59s
177:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 992ms	remaining: 4.58s
178:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 996ms	remaining: 4.57s
179:	learn: 0.0071429	test: 0.5145165	best: 0.4528726 (1)	total: 1s	remaining: 4.56s
180:	learn: 0.0071429	test: 0.514516

89:	learn: 0.0693848	test: 0.4883459	best: 0.4528726 (1)	total: 574ms	remaining: 5.8s
90:	learn: 0.0729562	test: 0.4877326	best: 0.4528726 (1)	total: 581ms	remaining: 5.81s
91:	learn: 0.0658134	test: 0.4951953	best: 0.4528726 (1)	total: 597ms	remaining: 5.89s
92:	learn: 0.0658134	test: 0.4883459	best: 0.4528726 (1)	total: 605ms	remaining: 5.9s
93:	learn: 0.0693848	test: 0.4945819	best: 0.4528726 (1)	total: 609ms	remaining: 5.87s
94:	learn: 0.0629232	test: 0.5020446	best: 0.4528726 (1)	total: 612ms	remaining: 5.83s
95:	learn: 0.0664946	test: 0.5088939	best: 0.4528726 (1)	total: 617ms	remaining: 5.81s
96:	learn: 0.0629232	test: 0.5026579	best: 0.4528726 (1)	total: 621ms	remaining: 5.78s
97:	learn: 0.0700661	test: 0.5026579	best: 0.4528726 (1)	total: 625ms	remaining: 5.75s
98:	learn: 0.0664946	test: 0.5095073	best: 0.4528726 (1)	total: 630ms	remaining: 5.73s
99:	learn: 0.0629232	test: 0.5157432	best: 0.4528726 (1)	total: 634ms	remaining: 5.71s
100:	learn: 0.0593518	test: 0.5020446	best: 0

192:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.14s	remaining: 4.78s
193:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.15s	remaining: 4.78s
194:	learn: 0.0035714	test: 0.5008178	best: 0.4528726 (1)	total: 1.15s	remaining: 4.76s
195:	learn: 0.0035714	test: 0.5008178	best: 0.4528726 (1)	total: 1.16s	remaining: 4.75s
196:	learn: 0.0035714	test: 0.5008178	best: 0.4528726 (1)	total: 1.17s	remaining: 4.77s
197:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.19s	remaining: 4.8s
198:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.19s	remaining: 4.79s
199:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.2s	remaining: 4.79s
200:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.2s	remaining: 4.78s
201:	learn: 0.0035714	test: 0.5076671	best: 0.4528726 (1)	total: 1.2s	remaining: 4.76s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4528726232
bestIteration = 1

Shrink model to first 2 i

## Backtesting:  Calculate PnL 

In [48]:
df_test = df_merged.loc[end_date:][[(f, ticker) for f in indicator_features+raw_features]]
for c in dt_features:
    df_test[c] = df_merged.loc[end_date:][c]
df_test.columns = df_test.columns.get_level_values(0)

for (dt, feats) in df_test[feature_names].iterrows():
    probs = model.predict_proba(pd.DataFrame([feats]))
    neg_class_prob, pos_class_prob = probs[0]
    print(probs[0])
    
    if pos_class_prob > 0.75:
        # buy
        pass
        
    elif neg_class_prob > 0.75:
        # sell
        pass
    
    break

[0.34814109 0.65185891]
